In [40]:
%pip install pysmt
%pip install z3-solver


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [41]:
from pysmt.typing import INT
from pysmt.shortcuts import *
from pysmt.shortcuts import BV, BVType, GE, Solver, Symbol


import itertools

In [42]:
from pysmt.typing import BVType
from pysmt.shortcuts import Symbol


def states(i):
    state = {}
    state['LFSR1'] = Symbol('LFSR1' + str(i), BVType(19))
    state['LFSR2'] = Symbol('LFSR2' + str(i), BVType(22))
    state['LFSR3'] = Symbol('LFSR3' + str(i), BVType(23))
    return state


def init(state):
    return And(state['LFSR1'] > 0,state['LFSR2'] > 0,state['LFSR3'] > 0)

def error(state):
    return Not(And(state['LFSR1'] > 0,state['LFSR2'] > 0,state['LFSR3'] > 0))

from pysmt.shortcuts import BVExtract, BVConcat, And, Or, Not, Equals

def transition(atual, prox):
    cBit1 = BVExtract(atual['LFSR1'], 8, 8)
    cBit2 = BVExtract(atual['LFSR2'], 10, 10)
    cBit3 = BVExtract(atual['LFSR3'], 10, 10)

    t1y = And(
        Equals(
            prox['LFSR1'], 
            BVConcat(
                BVExtract(atual['LFSR1'], 18, 18) ^ BVExtract(atual['LFSR1'], 17, 17) ^
                BVExtract(atual['LFSR1'], 16, 16) ^ BVExtract(atual['LFSR1'], 13, 13),
                BVExtract(atual['LFSR1'], 18, 1)
            )
        ),
        Or(cBit1 == cBit2, cBit1 == cBit3)
    )

    t1n = And(Equals(prox['LFSR1'], atual['LFSR1']), Not(Or(cBit1 == cBit2, cBit1 == cBit3)))

    t2y = And(
        Equals(
            prox['LFSR2'], 
            BVConcat(
                BVExtract(atual['LFSR2'], 21, 21) ^ BVExtract(atual['LFSR2'], 20, 20),
                BVExtract(atual['LFSR2'], 21, 1)
            )
        ),
        Or(cBit1 == cBit2, cBit2 == cBit3)
    )

    t2n = And(Equals(prox['LFSR2'], atual['LFSR2']), Not(Or(cBit1 == cBit2, cBit2 == cBit3)))

    t3y = And(
        Equals(
            prox['LFSR3'], 
            BVConcat(
                BVExtract(atual['LFSR3'], 22, 22) ^ BVExtract(atual['LFSR3'], 21, 21) ^
                BVExtract(atual['LFSR3'], 20, 20) ^ BVExtract(atual['LFSR3'], 7, 7),
                BVExtract(atual['LFSR3'], 22, 1)
            )
        ),
        Or(cBit3 == cBit2, cBit1 == cBit3)
    )

    t3n = And(Equals(prox['LFSR3'], atual['LFSR3']), Not(Or(cBit3 == cBit2, cBit1 == cBit3)))

    return And(Or(t1y, t1n), Or(t2y, t2n), Or(t3y, t3n))

def gera_sfots(init, error, trans, k):
    solver = Solver()

    estados = [states(i) for i in range(k)]

    # Add initial state assertion
    solver.add_assertion(init(estados[0]))

    # Add assertions to avoid the error state
    for i in range(k):
        solver.add_assertion(Not(error(estados[i])))

    # Add transition constraints
    for i in range(k - 1):
        solver.add_assertion(trans(estados[i], estados[i + 1]))

    # Check for satisfiability
    check = solver.solve()

    if check:
        print("Is solvable")
        m = solver.get_model()
        for i in range(k):
            print("Step", i + 1)
            for v in estados[i]:
                x = m[estados[i][v]]
                print(f"{v} = {format(x.constant_value(), f'0{x.bv_width()}b')}")
            print("--------------------------------")
    else:
        print("Not solvable")

gera_sfots(init,error,transition,10)

AssertionError: Start: 18 ; End: 1

In [ ]:
def states_PDR(i):
    state = {}

    state['LFSR1'] = BitVec('LFSR1',19)
    state['LFSR2'] = BitVec('LFSR2',22)
    state['LFSR3'] = BitVec('LFSR3',23)

    return state

# Nesta função não descrevi o P porque o estado de erro é a negação do init, logo como o P é a negação do estado de erro, P é igual ao init
def PDR(init,trans,error):
    
    solver = Solver()

    frames = [Not(init(states_PDR(0)))]
    k = 0

    while True:
        print("Iteração",k)
        bad = get_bad_cubes(error,frames,k,solver)

        if bad == None:
            if k > 0 and frames[k] == frames[k-1]:
                print("System is Safe")
                return None
            else:
                frames.append(False)
                k+=1
        else:
            block = bloqueio(bad,frames,k,trans,solver)
            if not block:
                print("System is Unsafe")
                return None


def get_bad_cubes(error,frames,k,solver):

    estado_atual = states_PDR(k)
    solver.push()
    for frame in frames:
        solver.add(Not(frame))
    solver.add(error(estado_atual))

    if solver.check() == sat:
        model = solver.model()
        cube = {var: model.eval(var, model_completion=True) for var in estado_atual.values()}
        solver.pop()
        return cube
    
    solver.pop()
    return None

def bloqueio(bad_cube,frames,k,trans,solver):

    for i in range(k,1,-1):
        solver.push()
        estado_atual = states_PDR(i-1)
        proximo_estado = states_PDR(i)

        solver.add(trans(estado_atual,proximo_estado))
        for j in range(i):
            solver.add(Not(frames[j]))
        cube = True
        for var,val in bad_cube.items():
            cube = And(cube,var != val)

        solver.add(cube)
        
        if solver.check() == unsat:
            solver.pop()
            print("Cubo bloqueado no frame",i)
            frames[i] = cube
            return True
        solver.pop()

    return False

PDR(init,transition,error)